In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("anthonytherrien/dog-vs-cat")

print("Path to dataset files:", path)

C:\Users\DELL\AppData\Roaming\Python\Python313\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\DELL\.cache\kagglehub\datasets\anthonytherrien\dog-vs-cat\versions\3


In [10]:
import os
root=os.path.join(path, "animals")


In [11]:
!pip install torchvision
from torchvision import datasets, transforms

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

dataset = datasets.ImageFolder(
    root=os.path.join(path, "animals"),
    transform=transform
)

print(dataset.classes)   # ['cats', 'dogs']
print(dataset.class_to_idx)  # {'cats': 0, 'dogs': 1}


   ---------------------------------------- 0.0/4.3 MB ? eta -:--:--
   ------- -------------------------------- 0.8/4.3 MB 4.8 MB/s eta 0:00:01
   ---------------- ----------------------- 1.8/4.3 MB 4.8 MB/s eta 0:00:01
   -------------------------- ------------- 2.9/4.3 MB 4.8 MB/s eta 0:00:01
   ------------------------------------ --- 3.9/4.3 MB 4.8 MB/s eta 0:00:01
   ---------------------------------------- 4.3/4.3 MB 4.6 MB/s eta 0:00:00
   ---------------------------------------- 0.0/113.8 MB ? eta -:--:--
   ---------------------------------------- 0.8/113.8 MB 4.6 MB/s eta 0:00:25
    --------------------------------------- 1.8/113.8 MB 4.8 MB/s eta 0:00:24
   - -------------------------------------- 2.9/113.8 MB 4.8 MB/s eta 0:00:23
   - -------------------------------------- 3.9/113.8 MB 4.8 MB/s eta 0:00:23
   - -------------------------------------- 5.0/113.8 MB 4.8 MB/s eta 0:00:23
   -- ------------------------------------- 6.0/113.8 MB 4.8 MB/s eta 0:00:23
   -- ------


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


['cat', 'dog']
{'cat': 0, 'dog': 1}


In [12]:
from torch.utils.data import random_split, DataLoader

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size

train_ds, val_ds = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32)


In [18]:
import tensorflow as tf

train_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(path, "animals"),
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    os.path.join(path, "animals"),
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(224, 224),
    batch_size=32
)


Found 1000 files belonging to 2 classes.
Using 800 files for training.
Found 1000 files belonging to 2 classes.
Using 200 files for validation.


In [19]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense, Conv2D, MaxPooling2D, Flatten


In [30]:
# Making a CNN model
model= Sequential()

# CNN Layers
model.add(Conv2D(32,kernel_size=(3,3),padding='valid',activation='relu',input_shape=(224,224,3))) #first layer 32 filters 3
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(64,kernel_size=(3,3),padding='valid',activation='relu')) #second  layer 64 filters 
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Conv2D(128,kernel_size=(3,3),padding='valid',activation='relu')) #Third layer 128 filters 3
model.add(MaxPooling2D(pool_size=(2,2),strides=2,padding='valid'))

model.add(Flatten()) #Flattening 

# ANN Layers
model.add(Dense(128,activation='relu')) # Fourth layer 
model.add(Dense(64,activation='relu')) # Fifth layer 
model.add(Dense(1,activation='softmax')) # output layer 


In [31]:
model.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_8 (Conv2D)               │ (None, 222, 222, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_8 (MaxPooling2D)  │ (None, 111, 111, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_9 (Conv2D)               │ (None, 109, 109, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_9 (MaxPooling2D)  │ (None, 54, 54, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_10 (Conv2D)              │ (None, 52, 52, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_10 (MaxPooling2D) │ (None, 26, 26, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 86528)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 128)            │    11,075,712 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,177,281 (42.64 MB)

 Trainable params: 11,177,281 (42.64 MB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)


In [39]:
history = model.fit(
    train_ds,
    epochs=10,
    validation_data=val_ds
)


Epoch 1/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 672ms/step - accuracy: 0.5013 - loss: 3.9799e-05 - val_accuracy: 0.4950 - val_loss: 1.1319
Epoch 2/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 16s 648ms/step - accuracy: 0.5013 - loss: 2.9720e-05 - val_accuracy: 0.4950 - val_loss: 1.1502
Epoch 3/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 677ms/step - accuracy: 0.5013 - loss: 2.3686e-05 - val_accuracy: 0.4950 - val_loss: 1.1686
Epoch 4/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 24s 964ms/step - accuracy: 0.5013 - loss: 1.9410e-05 - val_accuracy: 0.4950 - val_loss: 1.1884
Epoch 5/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 20s 803ms/step - accuracy: 0.5013 - loss: 1.5744e-05 - val_accuracy: 0.4950 - val_loss: 1.2070
Epoch 6/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 32s 1s/step - accuracy: 0.5013 - loss: 1.3170e-05 - val_accuracy: 0.4950 - val_loss: 1.2232
Epoch 7/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 41s 1s/step - accuracy: 0.5013 - loss: 1.1186e-05 - val_accuracy: 0.4950 - val_loss: 1.2393
Epoch 8/10
25/25 ━━━━━━━━━━━━━━━━━━━━ 17s 661ms/step - accuracy: 0.5013 - l

In [40]:
# Testing on an image
import tensorflow as tf
import numpy as np
from PIL import Image

def predict_image(img_path, model):
    # Load image
    img = Image.open(img_path).convert("RGB")
    
    # Resize (must match training size)
    img = img.resize((224, 224))
    
    # Convert to array
    img_array = np.array(img) / 255.0  # normalize
    
    # Add batch dimension
    img_array = np.expand_dims(img_array, axis=0)  # (1, 224, 224, 3)
    
    # Predict
    prediction = model.predict(img_array)
    
    # Binary classification output
    if prediction[0][0] > 0.5:
        return "Dog 🐶", prediction[0][0]
    else:
        return "Cat 🐱", prediction[0][0]

image=r"C:\Users\DELL\OneDrive\Desktop\download (1).jpg"
predict_image(image,model)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 66ms/step


('Dog 🐶', np.float32(1.0))